# Import libraries

In [13]:
# Earth engine code
import ee

In [14]:
# log in to earth engine
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

In [23]:
import geemap
from pathlib import Path

import geopandas as gpd

import multiprocess
from functools import partial

from tqdm import tqdm

In [4]:
tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

# Define variables

In [6]:
# path where the images will be downloaded to
outputPath = Path('MyOutputPath')

In [6]:
# projection to be used for the output
projectCRS = 'epsg:3857'

In [ ]:
# which date do we want to download images for
date = '2018-12-31'

In [6]:
# path to the geojson containing the region of interest
pathToGeoJson = 'myGeoJson.geojson'

In [6]:
# number of cores to use (parallel processing)
nbCores = 10

# Setup functions to run

In [6]:
# convert the geojson file to a dissolved ee geometry
shape = gpd.read_file(pathToGeoJson).dissolve()
geom = geemap.geopandas_to_ee(gpd.GeoDataFrame(shape.geometry)).geometry()

In [18]:
# create a covering grid over the geometry, save the grid for futur references
# set override to true when you call this to regenerate the grid
def GetCoveringGrid(outputDataFolder, geom, crs, resolution, override=False):
    coveringGridPath = outputDataFolder/'Shapefiles'/'CoveringGrid.geojson'
    coveringGridPath.parent.mkdir(exist_ok=True, parents=True)

    if (not coveringGridPath.exists()) or override:
        grid = geom.coveringGrid(crs, resolution)
        gridSize = grid.size().getInfo()
        indexes = ee.List(grid.aggregate_array('system:index'))
        ids = ee.List.sequence(1, gridSize)
        idByIndex = ee.Dictionary.fromLists(indexes, ids)
        grid = grid.map(lambda feature: feature.set('ItemID', idByIndex.get(feature.get('system:index'))))
        geemap.ee_to_geojson(grid, coveringGridPath.as_posix())
        

    coveringGrid = geemap.geojson_to_ee(coveringGridPath.as_posix())
    
    return coveringGrid

In [19]:
# sentinel-1 collection
Sen1Collection = ee.ImageCollection("COPERNICUS/S1_GRD")


filters = [
        ee.Filter.listContains("transmitterReceiverPolarisation", "VV"),\
        ee.Filter.listContains("transmitterReceiverPolarisation", "VH"),\
        ee.Filter.equals("instrumentMode", "IW")\
    ]
    
Sen1Collection = Sen1Collection.filter(filters)

In [21]:
# download a single chip element
def DownloadChip(element_index, image, outputFolder, grid, date, crs):
    feature = ee.Feature(grid.get(element_index))
    RoI = feature.geometry()
    
    chipID = feature.get('ItemID').getInfo()
    
    
    ##! This is the target folder, if you don't like the structure, change it here
    targetFolder = (outputFolder/str(chipID)/'S1')
    if(not targetFolder.exists()):
        targetFolder.mkdir(parents=True)
    
    ##! same for the chip ID, change here 
    chipPath = targetFolder/f"{chipID}_{date}.tif"
    
    if chipPath.exists():
        return
    
    chipPath = chipPath.as_posix()
    
    geemap.download_ee_image(image, chipPath, crs=crs, region=RoI, shape=(256, 256))

In [22]:
# download all the chips for a given date
def DownloadAllChipsForDate(coveringGridList, listSize, crs, image, date, outputFolder):
    with multiprocess.Pool(nbCores) as pool:
        pool.map(partial(DownloadChip, image=image, outputFolder=outputFolder, grid=coveringGridList, date=date, crs=crs), range(listSize))

In [23]:
# Get image for date and boundary
def GetImage(date, boundary):
    
    startDate = ee.Date(date).advance(k, 'Day')
    filteredCollection = Sen1Collection.filterBounds(boundary).filterDate(startDate, startDate.advance(1, 'Day'))
    
    imageBoundaries = filteredCollection.geometry().dissolve()
    image = filteredCollection.mosaic()
    
    crs = filteredCollection.first().select('VV').projection().crs().getInfo()
    
    image = image.setDefaultProjection(crs)
    
    return image, imageBoundaries

In [24]:
# download all chips for a given geometry and date
def DownloadChipsForGeomDate(geom, date, outputPath):
    image, imageBoundaries = GetImage(date, geom)
    coveringGrid = GetCoveringGrid(outputPath, geom, projectCRS, 10*256, False)
    coveringGrid = coveringGrid.map(lambda feat: feat.set('touch', feat.geometry().intersects(bound,1))).filter(ee.Filter.eq('touch', True))

    listSize = coveringGrid.size().getInfo()
    coveringGridList = coveringGrid.toList(listSize)

    DownloadAllChipsForDate(coveringGridList, listSize, projectCRS, image, date_k, outputLocation_k)

# Download all images

In [24]:
#runs all the code
DownloadChipsForGeomDate(geom, date, outputPath)